In [2]:
import pandas as pd
df = pd.read_csv("sales.csv")
df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640840 entries, 0 to 640839
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Unnamed: 0           640840 non-null  int64 
 1   store_ID             640840 non-null  int64 
 2   day_of_week          640840 non-null  int64 
 3   date                 640840 non-null  object
 4   nb_customers_on_day  640840 non-null  int64 
 5   open                 640840 non-null  int64 
 6   promotion            640840 non-null  int64 
 7   state_holiday        640840 non-null  object
 8   school_holiday       640840 non-null  int64 
 9   sales                640840 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 48.9+ MB


,Unnamed: 0,store_ID,day_of_week,nb_customers_on_day,open,promotion,school_holiday,sales
count,640840.000000,640840.000000,640840.000000,640840.000000,640840.000000,640840.000000,640840.000000,640840.000000
mean,355990.675084,558.211348,4.000189,633.398577,0.830185,0.381718,0.178472,5777.469011
std,205536.290268,321.878521,1.996478,464.094416,0.375470,0.485808,0.382910,3851.338083
min,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,178075.750000,280.000000,2.000000,405.000000,1.000000,0.000000,0.000000,3731.000000
50%,355948.500000,558.000000,4.000000,609.000000,1.000000,0.000000,0.000000,5746.000000
75%,533959.250000,837.000000,6.000000,838.000000,1.000000,1.000000,0.000000,7860.000000
max,712044.000000,1115.000000,7.000000,5458.000000,1.000000,1.000000,1.000000,41551.000000


In [3]:
df['date'] = pd.to_datetime(df['date'])

# Example feature engineering:
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['is_weekend'] = df['day_of_week'].isin([6, 7]).astype(int)

#df.describe()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640840 entries, 0 to 640839
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Unnamed: 0           640840 non-null  int64         
 1   store_ID             640840 non-null  int64         
 2   day_of_week          640840 non-null  int64         
 3   date                 640840 non-null  datetime64[ns]
 4   nb_customers_on_day  640840 non-null  int64         
 5   open                 640840 non-null  int64         
 6   promotion            640840 non-null  int64         
 7   state_holiday        640840 non-null  object        
 8   school_holiday       640840 non-null  int64         
 9   sales                640840 non-null  int64         
 10  month                640840 non-null  int32         
 11  year                 640840 non-null  int32         
 12  is_weekend           640840 non-null  int64         
dtypes: datetime64[

In [4]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['sales'])
y = df['sales']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:

# Drop 'date' from both sets because it's not directly useful as a raw feature in most machine learning models. 
X_train_prep = X_train.drop('date', axis=1)
X_val_prep = X_val.drop('date', axis=1)

# Sample 50,000 rows from the training set
X_sample = X_train_prep.sample(n=50000, random_state=42)
y_sample = y_train.loc[X_sample.index]


In [15]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
import numpy as np

# === Log-transform the target ===
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

# === Encode categorical features ===
X_sample_encoded = X_sample.copy()
X_val_prep_encoded = X_val_prep.copy()

le = LabelEncoder()
X_sample_encoded['state_holiday'] = le.fit_transform(X_sample['state_holiday'])
X_val_prep_encoded['state_holiday'] = le.transform(X_val_prep['state_holiday'])

# === Define the model with tuned hyperparameters ===
model = XGBRegressor(
    n_estimators=200,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# === Fit the model with early stopping on log-transformed target ===
# We need to use y_sample_log instead of y_train_log since we're using X_sample_encoded
y_sample_log = np.log1p(y_sample)

model.fit(
    X_sample_encoded,
    y_sample_log,
    eval_set=[(X_val_prep_encoded, y_val_log)],
    verbose=True
)

# === Predict and invert log1p ===
y_pred_log = model.predict(X_val_prep_encoded)
y_pred = np.expm1(y_pred_log)  # Convert back to normal scale
y_val_original = np.expm1(y_val_log)  # Ground truth in normal scale

# === Evaluate ===
r2 = r2_score(y_val_original, y_pred)
print("Validation R² Score:", r2)


[0]	validation_0-rmse:3.14781
[1]	validation_0-rmse:2.99110
[2]	validation_0-rmse:2.84456
[3]	validation_0-rmse:2.70317
[4]	validation_0-rmse:2.57104
[5]	validation_0-rmse:2.44338
[6]	validation_0-rmse:2.32300
[7]	validation_0-rmse:2.20774
[8]	validation_0-rmse:2.09828
[9]	validation_0-rmse:1.99434
[10]	validation_0-rmse:1.89569
[11]	validation_0-rmse:1.80240
[12]	validation_0-rmse:1.71340
[13]	validation_0-rmse:1.62894
[14]	validation_0-rmse:1.54924
[15]	validation_0-rmse:1.47308
[16]	validation_0-rmse:1.40081
[17]	validation_0-rmse:1.33216
[18]	validation_0-rmse:1.26755
[19]	validation_0-rmse:1.20609
[20]	validation_0-rmse:1.14736
[21]	validation_0-rmse:1.09166
[22]	validation_0-rmse:1.03924
[23]	validation_0-rmse:0.98911
[24]	validation_0-rmse:0.94188
[25]	validation_0-rmse:0.89717
[26]	validation_0-rmse:0.85440
[27]	validation_0-rmse:0.81424
[28]	validation_0-rmse:0.77580
[29]	validation_0-rmse:0.73969
[30]	validation_0-rmse:0.70556
[31]	validation_0-rmse:0.67360
[32]	validation_0-

In [7]:
with open("r2-XG.txt", "w") as f:
    f.write(str(r2))


In [8]:
feature_importance = pd.DataFrame({
    'Feature': X_sample_encoded.columns,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)
print("\nTop 10 most important features:")
print(feature_importance.head(10))


Top 10 most important features:
                Feature  Importance
3   nb_customers_on_day    0.433052
4                  open    0.394651
5             promotion    0.100695
1              store_ID    0.018685
2           day_of_week    0.014684
8                 month    0.008845
10           is_weekend    0.008661
6         state_holiday    0.006622
9                  year    0.005524
7        school_holiday    0.004579
